In [1]:
source("http://bioconductor.org/biocLite.R")
###to be done once
#install bioclite
#biocLite()
#install top go
#biocLite("topGO")

Bioconductor version 3.2 (BiocInstaller 1.20.0), ?biocLite for help


In [2]:
library(topGO)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, grep, grepl,
    intersect, is.unsorted, lapply, lengths, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unlist, unsplit

Loading required package: graph
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view wi


groupGOTerms: 	GOBPTerm, GOMFTerm, GOCCTerm environments built.



Attaching package: ‘topGO’

The following object is masked from ‘package:IRanges’:

    members



In [3]:
xpclr_ana_dir <- "~/vervet_project/analyses/20150528_UG_xpclr/_data"

In [4]:
vervet_drive_dir <- "~/Akademisches/Projects/VervetPopgen"

## Example
from http://bioconductor.org/packages/release/bioc/vignettes/topGO/inst/doc/topGO.pdf

In [7]:
#load example data
library(ALL)
data(ALL)
#geneList is a list of genes with associated p-values
data(geneList)

In [15]:
#I think this is the go database
affyLib <- paste(annotation(ALL), "db", sep = ".") #name (string)
#biocLite(affyLib)
library(package = affyLib, character.only = TRUE)

Loading required package: org.Hs.eg.db




In [23]:
#this creates a go to gene mapping (done automatically in new(..))
#geneToGo <- annFUN.db(whichOnto="BP",affyLib=affyLib)

In [26]:
#create the go object
#topDiffGenes is a function that selects significant p-values
#nodeSize = 10 includes gene categories with less than 10 genes
sampleGOdata <- new("topGOdata",
                    description = "Simple session", ontology = "BP",
                    allGenes = geneList, geneSel = topDiffGenes,
                    nodeSize = 10,
                    annot = annFUN.db, affyLib = affyLib)


Building most specific GOs .....	( 1464 GO terms found. )

Build GO DAG topology ..........	( 4104 GO terms and 9644 relations. )

Annotating nodes ...............	( 310 genes annotated to the GO terms. )


In [28]:
#classic candidate vs background test
resultFisher <- runTest(sampleGOdata, algorithm = "classic", statistic = "fisher")
#run Kolmogorov Smirnov test on expression p-vlaues of all genes
resultKS <- runTest(sampleGOdata, algorithm = "classic", statistic = "ks")
#accounts for gene overlap
resultKS.elim <- runTest(sampleGOdata, algorithm = "elim", statistic = "ks")


			 -- Classic Algorithm -- 

		 the algorithm is scoring 973 nontrivial nodes
		 parameters: 
			 test statistic:  fisher 

			 -- Classic Algorithm -- 

		 the algorithm is scoring 1043 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 score order:  increasing 

			 -- Elim Algorithm -- 

		 the algorithm is scoring 1043 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 cutOff:  0.01 
			 score order:  increasing 

	 Level 17:	1 nodes to be scored	(0 eliminated genes)

	 Level 16:	1 nodes to be scored	(0 eliminated genes)

	 Level 15:	1 nodes to be scored	(0 eliminated genes)

	 Level 14:	8 nodes to be scored	(0 eliminated genes)

	 Level 13:	17 nodes to be scored	(0 eliminated genes)

	 Level 12:	31 nodes to be scored	(0 eliminated genes)

	 Level 11:	33 nodes to be scored	(10 eliminated genes)

	 Level 10:	52 nodes to be scored	(20 eliminated genes)

	 Level 9:	88 nodes to be scored	(20 eliminated genes)

	 Level 8:	126 nodes to be scored	(29 eliminated gen

In [29]:
allRes <- GenTable(sampleGOdata, classicFisher = resultFisher,
  classicKS = resultKS, elimKS = resultKS.elim,
  orderBy = "elimKS", ranksOf = "classicFisher", topNodes = 10)

In [30]:
allRes

,GO.ID,Term,Annotated,Significant,Expected,Rank in classicFisher,classicFisher,classicKS,elimKS
1,GO:0007067,mitotic nuclear division,173,17,25.67,968,0.99839,3.2e-09,3.2e-09
2,GO:0051301,cell division,161,18,23.89,915,0.97969,4.9e-08,4.4e-07
3,GO:0032465,regulation of cytokinesis,13,3,1.93,334,0.30050,0.00120,0.0012
4,GO:0007059,chromosome segregation,77,5,11.43,960,0.99700,0.00126,0.0013
5,GO:0051726,regulation of cell cycle,128,17,18.99,636,0.78999,0.00014,0.0013
6,GO:0045931,positive regulation of mitotic cell cycl...,40,5,5.94,592,0.74532,0.00175,0.0017
7,GO:0033077,T cell differentiation in thymus,10,6,1.48,8,0.00104,0.00186,0.0019
8,GO:0002757,immune response-activating signal transd...,11,7,1.63,2,0.00021,0.00208,0.0021
9,GO:0042108,positive regulation of cytokine biosynth...,18,6,2.67,107,0.03527,0.00252,0.0025
10,GO:0090068,positive regulation of cell cycle proces...,38,3,5.64,845,0.94667,0.00283,0.0028


## Vervet with standard go annotations

In [17]:
geneScores <- read.table('/home/GMI/hannes.svardal/vervet_project/analyses/20150528_UG_xpclr/_data/rms_zscore_gene_mean.tsv',header=TRUE,row.names=1)
geneScoresMax <- read.table('/home/GMI/hannes.svardal/vervet_project/analyses/20150528_UG_xpclr/_data/rms_zscores_gene_max_exons.tsv',header=TRUE,row.names=1)

In [18]:
#human whole genome annotation
library("org.Hs.eg.db")

In [19]:
selFun <- function(col) {
    return(col>quantile(col,0.99))
}

In [20]:
geneScoresVec <- as.vector(geneScoresMax[["score"]])
names(geneScoresVec) <- row.names(geneScores)

In [21]:
humanGOdata <- new("topGOdata",
                    description = "Vervet xpclr RMS gene mean scores", ontology = "BP",
                    allGenes = geneScoresVec, geneSel = selFun,
                    nodeSize = 10,
                    annot = annFUN.org, mapping = "org.Hs.eg.db", ID = "symbol") #human go


Building most specific GOs .....	( 10308 GO terms found. )

Build GO DAG topology ..........	( 13955 GO terms and 33446 relations. )

Annotating nodes ...............	( 13319 genes annotated to the GO terms. )


In [22]:
#classic candidate vs background test
resultFisher <- runTest(humanGOdata, algorithm = "classic", statistic = "fisher")
#run Kolmogorov Smirnov test on expression p-vlaues of all genes
#resultKS <- runTest(humanGOdata, algorithm = "classic", statistic = "ks", scoreOrder='decreasing')
#accounts for gene overlap
#resultKS.elim <- runTest(humanGOdata, algorithm = "elim", statistic = "ks", scoreOrder='decreasing')
resultKS.weight01 <- runTest(humanGOdata, algorithm = "weight01", statistic = "ks", scoreOrder='decreasing')


			 -- Classic Algorithm -- 

		 the algorithm is scoring 2776 nontrivial nodes
		 parameters: 
			 test statistic:  fisher 

			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 5777 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 score order:  decreasing 

	 Level 18:	1 nodes to be scored	(0 eliminated genes)

	 Level 17:	5 nodes to be scored	(0 eliminated genes)

	 Level 16:	19 nodes to be scored	(11 eliminated genes)

	 Level 15:	47 nodes to be scored	(89 eliminated genes)

	 Level 14:	115 nodes to be scored	(309 eliminated genes)

	 Level 13:	199 nodes to be scored	(796 eliminated genes)

	 Level 12:	322 nodes to be scored	(2021 eliminated genes)

	 Level 11:	503 nodes to be scored	(3767 eliminated genes)

	 Level 10:	663 nodes to be scored	(5362 eliminated genes)

	 Level 9:	817 nodes to be scored	(7585 eliminated genes)

	 Level 8:	843 nodes to be scored	(9259 eliminated genes)

	 Level 7:	839 nodes to be scored	(10417 eliminated genes)

	 Level 6:	660 nodes

In [11]:
resultFisher.weight01 <- runTest(humanGOdata, algorithm = "weight01", statistic = "fisher")


			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 2776 nontrivial nodes
		 parameters: 
			 test statistic:  fisher 

	 Level 16:	3 nodes to be scored	(0 eliminated genes)

	 Level 15:	17 nodes to be scored	(0 eliminated genes)

	 Level 14:	44 nodes to be scored	(53 eliminated genes)

	 Level 13:	75 nodes to be scored	(386 eliminated genes)

	 Level 12:	115 nodes to be scored	(1490 eliminated genes)

	 Level 11:	180 nodes to be scored	(3184 eliminated genes)

	 Level 10:	274 nodes to be scored	(4508 eliminated genes)

	 Level 9:	355 nodes to be scored	(6545 eliminated genes)

	 Level 8:	400 nodes to be scored	(8141 eliminated genes)

	 Level 7:	423 nodes to be scored	(9547 eliminated genes)

	 Level 6:	372 nodes to be scored	(11095 eliminated genes)

	 Level 5:	282 nodes to be scored	(11885 eliminated genes)

	 Level 4:	164 nodes to be scored	(12537 eliminated genes)

	 Level 3:	51 nodes to be scored	(12802 eliminated genes)

	 Level 2:	20 nodes to be scored	(12993 eliminated

In [13]:
resultFisher


Description: Vervet xpclr RMS gene mean scores 
Ontology: BP 
'classic' algorithm with the 'fisher' test
5777 GO terms scored: 106 terms with p < 0.01
Annotation data:
    Annotated genes: 13319 
    Significant genes: 197 
    Min. no. of genes annotated to a GO: 10 
    Nontrivial nodes: 2776 

In [14]:
resultKS.weight01


Description: Vervet xpclr RMS gene mean scores 
Ontology: BP 
'weight01' algorithm with the 'ks' test
5777 GO terms scored: 167 terms with p < 0.01
Annotation data:
    Annotated genes: 13319 
    Significant genes: 197 
    Min. no. of genes annotated to a GO: 10 
    Nontrivial nodes: 5777 

In [23]:
allRes <- GenTable(humanGOdata,#classicFisher = resultFisher, ranksOf = "classicFisher",
                 #  weight01Fisher = resultFisher.weight01,
       weight01KS = resultKS.weight01,
  orderBy = "weight01KS", topNodes = 167)#

In [24]:
allRes

,GO.ID,Term,Annotated,Significant,Expected,weight01KS
1,GO:0000122,negative regulation of transcription fro...,660,21,9.76,4.2e-17
2,GO:0045944,positive regulation of transcription fro...,928,24,13.73,4.3e-14
3,GO:0009952,anterior/posterior pattern specification,191,12,2.83,5.0e-12
4,GO:0016032,viral process,685,13,10.13,9.5e-10
5,GO:0045893,"positive regulation of transcription, DN...",1297,32,19.18,5.4e-09
6,GO:0006406,mRNA export from nucleus,70,2,1.04,1.3e-06
7,GO:0051301,cell division,587,14,8.68,7.3e-06
8,GO:0006357,regulation of transcription from RNA pol...,1590,42,23.52,7.6e-06
9,GO:0006397,mRNA processing,397,12,5.87,8.7e-06
10,GO:0006336,DNA replication-independent nucleosome a...,33,1,0.49,1.0e-05


In [30]:
install.packages("xlsx", repos="http://cran.us.r-project.org")

also installing the dependencies ‘rJava’, ‘xlsxjars’




The downloaded source packages are in
	‘/tmp/RtmpThbomn/downloaded_packages’


In [31]:
library(xlsx)

Loading required package: rJava
Loading required package: xlsxjars


In [34]:
write.xlsx(allRes[1:50,],path.expand(paste(vervet_drive_dir,"share","xpclr_rms_exon_mean_topgo_KSweight01_top50.xlsx",sep='/')))

In [60]:
geneScoresVec[grep("^LOC",names(geneScoresVec))]

named numeric(0)

In [37]:
allRes[1,2]

[1] "negative regulation of transcription fro..."

### create input for Cytoscape Enrichment Map
info here: http://www.baderlab.org/Software/EnrichmentMap/UserManual#Generic_results_files

#### GMT file

In [35]:
go_to_gene <- annFUN.org("BP",feasibleGenes=names(geneScoresVec),mapping="org.Hs.eg.db",ID = "symbol")

In [264]:
length(go_to_gene)

[1] 10308

In [61]:
go_to_name <- select(GO.db,keys=names(go_to_gene),columns=c('TERM','DEFINITION'))
rownames(go_to_name) <- go_to_name[,'GOID']

'select()' returned 1:1 mapping between keys and columns


In [218]:
vecToStr <- function(vec) {
    return(paste(vec,collapse='\t'))#
}

In [219]:
go_to_gene_df <- data.frame(names(go_to_gene))
colnames(go_to_gene_df) <- "GOID"
#add a gene column with list of genes to data frame
go_to_gene_df$genes <- lapply(go_to_gene,vecToStr)
#this conversion is necessary so that write table does not complain that a data frame column is of type list
go_to_gene_df <- data.frame(lapply(go_to_gene_df, as.character), stringsAsFactors=FALSE)

In [220]:
#gmt format (GSEA)
gmt_df <- merge(go_to_name[,c("GOID","TERM")], go_to_gene_df, by="GOID")

In [ ]:
gmt_df

In [267]:
write.table(gmt_df,paste(vervet_drive_dir,"Cytoscape","topgo_go_to_gene.gmt",sep='/'),quote=FALSE,row.names=FALSE,col.names=FALSE,sep='\t')

#### Generic results file
http://www.baderlab.org/Software/EnrichmentMap/UserManual#Generic_results_files

In [284]:
resultKS.weight01


Description: Vervet xpclr RMS gene mean scores 
Ontology: BP 
'weight01' algorithm with the 'ks' test
5777 GO terms scored: 167 terms with p < 0.01
Annotation data:
    Annotated genes: 13319 
    Significant genes: 197 
    Min. no. of genes annotated to a GO: 10 
    Nontrivial nodes: 5777 

In [229]:
resultKS.weight01


Description: Vervet xpclr RMS gene mean scores 
Ontology: BP 
'weight01' algorithm with the 'ks' test
5777 GO terms scored: 167 terms with p < 0.01
Annotation data:
    Annotated genes: 13319 
    Significant genes: 145 
    Min. no. of genes annotated to a GO: 10 
    Nontrivial nodes: 5777 

In [285]:
xpclr_rms_enrich_results <- GenTable(humanGOdata, weight01KS = resultKS.weight01, orderBy = "weight01KS", topNodes = 167)

In [271]:
generic_result <- xpclr_rms_enrich_results[,c("GO.ID","Term","weight01KS")]

In [250]:
#generic_result$FDR = NA

In [251]:
#generic_result$Phenotype = NA

In [272]:
colnames(generic_result) <- c("GO.ID","Description","p.Val")

In [278]:
write.table(generic_result,paste(vervet_drive_dir,"Cytoscape","topgo_xpclr_rms_generic_results_file.tsv",sep='/'),row.names=FALSE,quote=FALSE,sep='\t')

In [277]:
length(generic_result)

[1] 3

### Vervet genes with phylo conscores 

to test whether we get the same results as with the xpclr scores

In [26]:
geneScores <- read.table('/home/GMI/hannes.svardal/vervet_project/analyses/20150528_UG_xpclr/_data/conscore_mean_per_gen.tsv',header=TRUE,row.names=1)

In [27]:
geneScoresVec <- as.vector(geneScores[["conscore"]])
names(geneScoresVec) <- row.names(geneScores)

In [28]:
humanGOdata <- new("topGOdata",
                    description = "Vervet xpclr RMS gene mean scores", ontology = "BP",
                    allGenes = geneScoresVec, geneSel = selFun,
                    nodeSize = 10,
                    annot = annFUN.org, mapping = "org.Hs.eg.db", ID = "symbol")


Building most specific GOs .....	( 10260 GO terms found. )

Build GO DAG topology ..........	( 13910 GO terms and 33340 relations. )

Annotating nodes ...............	( 13054 genes annotated to the GO terms. )


In [29]:
resultKS.weight01 <- runTest(humanGOdata, algorithm = "weight01", statistic = "ks", scoreOrder='decreasing')


			 -- Weight01 Algorithm -- 

		 the algorithm is scoring 5746 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 score order:  decreasing 

	 Level 18:	1 nodes to be scored	(0 eliminated genes)

	 Level 17:	5 nodes to be scored	(0 eliminated genes)

	 Level 16:	19 nodes to be scored	(11 eliminated genes)

	 Level 15:	47 nodes to be scored	(88 eliminated genes)

	 Level 14:	113 nodes to be scored	(307 eliminated genes)

	 Level 13:	197 nodes to be scored	(791 eliminated genes)

	 Level 12:	320 nodes to be scored	(1987 eliminated genes)

	 Level 11:	498 nodes to be scored	(3714 eliminated genes)

	 Level 10:	659 nodes to be scored	(5287 eliminated genes)

	 Level 9:	814 nodes to be scored	(7459 eliminated genes)

	 Level 8:	839 nodes to be scored	(9078 eliminated genes)

	 Level 7:	832 nodes to be scored	(10211 eliminated genes)

	 Level 6:	659 nodes to be scored	(11242 eliminated genes)

	 Level 5:	443 nodes to be scored	(11814 eliminated genes)

	 Level 4:	217 nodes to be

In [30]:
allResConscores <- GenTable(humanGOdata,#classicFisher = resultFisher, ranksOf = "classicFisher",
                 #  weight01Fisher = resultFisher.weight01,
       weight01KS = resultKS.weight01,
  orderBy = "weight01KS", topNodes = 167)#

In [33]:
sum(allResConscores["weight01KS"]<0.01)

[1] 41

Conclusion: The conservation scores only have 41 pvalues below 0.01 (as compared 167 for the xpclr scores)
    The categories don't have extremely strong overlap.
Here it is mainly medium-sized categories that are enriched.

## Vervet HIV categories

!!! THIS DOES NOT WORK. topGO needs real go-categories.

In [164]:
#geneID2GO <-  readMappings(file = system.file("examples/geneid2go.map", package = "topGO"))
geneID2GO <- readMappings(file = '/home/GMI/hannes.svardal/vervet_project/data/gene_associations/gene_to_hiv_r100_20151130_topgo.map')

In [86]:
#geneScoresVecTest = runif(length(geneID2GO))
#names(geneScoresVecTest) = names(geneID2GO)

In [165]:
hivGOdata <- new("topGOdata",
                    description = "Vervet xpclr RMS gene mean scores tested against hiv categories", ontology = "MF",
                    allGenes = geneScoresVec, geneSel = selFun,
                 annot = annFUN.gene2GO,
                     gene2GO = geneID2GO) #ncbi hiv categories


Building most specific GOs .....	( 0 GO terms found. )

Build GO DAG topology ..........	( 0 GO terms and 0 relations. )


ERROR: Error in sapply(adj(dag, nodes(dag)), length): error in evaluating the argument 'X' in selecting a method for function 'sapply': Error in if (is.na(index) || index < 0 || index > length(nd)) stop("vertex is not in graph: ",  : 
  missing value where TRUE/FALSE needed
Calls: adj -> adj



In [161]:
#classic candidate vs background test
resultFisher <- runTest(hivGOdata, algorithm = "classic", statistic = "fisher")
#run Kolmogorov Smirnov test on expression p-vlaues of all genes
resultKS <- runTest(hivGOdata, algorithm = "classic", statistic = "ks", scoreOrder='decreasing')
#accounts for gene overlap
resultKS.elim <- runTest(hivGOdata, algorithm = "elim", statistic = "ks", scoreOrder='decreasing')
resultKS.weight01 <- runTest(hivGOdata, algorithm = "weight01", statistic = "ks", scoreOrder='decreasing')


			 -- Classic Algorithm -- 

		 the algorithm is scoring 54 nontrivial nodes
		 parameters: 
			 test statistic:  fisher 

			 -- Classic Algorithm -- 

		 the algorithm is scoring 249 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 score order:  decreasing 

			 -- Elim Algorithm -- 

		 the algorithm is scoring 249 nontrivial nodes
		 parameters: 
			 test statistic:  ks 
			 cutOff:  0.01 
			 score order:  decreasing 

	 Level 13:	1 nodes to be scored	(0 eliminated genes)

	 Level 12:	2 nodes to be scored	(0 eliminated genes)

	 Level 11:	8 nodes to be scored	(0 eliminated genes)

	 Level 10:	7 nodes to be scored	(217 eliminated genes)

	 Level 9:	13 nodes to be scored	(217 eliminated genes)

	 Level 8:	22 nodes to be scored	(217 eliminated genes)

	 Level 7:	35 nodes to be scored	(217 eliminated genes)

	 Level 6:	49 nodes to be scored	(219 eliminated genes)

	 Level 5:	43 nodes to be scored	(318 eliminated genes)

	 Level 4:	40 nodes to be scored	(458 eliminated g

In [162]:
allRes <- GenTable(hivGOdata,classicFisher = resultFisher, ranksOf = "classicFisher",
  classicKS = resultKS, elimKS = resultKS.elim,
  orderBy = "elimKS", topNodes = 50)#

In [163]:
allRes

,GO.ID,Term,Annotated,Significant,Expected,Rank in classicFisher,classicFisher,classicKS,elimKS
1,GO:0000062,fatty-acyl-CoA binding,199,5,2.14,4,0.051,1.4e-09,1.4e-09
2,GO:0000064,L-ornithine transmembrane transporter ac...,182,4,1.95,12,0.121,1.3e-06,1.3e-06
3,GO:0000016,lactase activity,101,2,1.08,33,0.296,0.00010,0.00010
4,GO:0000309,nicotinamide-nucleotide adenylyltransfer...,7,0,0.08,54,1.000,0.00015,0.00015
5,GO:0000100,S-methylmethionine transmembrane transpo...,35,1,0.38,36,0.318,0.00027,0.00027
6,GO:0000250,lanosterol synthase activity,13,1,0.14,13,0.131,0.00043,0.00043
7,GO:0000285,1-phosphatidylinositol-3-phosphate 5-kin...,6,0,0.06,55,1.000,0.02280,0.02280
8,GO:0016307,phosphatidylinositol phosphate kinase ac...,6,0,0.06,56,1.000,0.02280,0.02280
9,GO:0000257,nitrilase activity,3,0,0.03,57,1.000,0.05528,0.05528
10,GO:0016815,"hydrolase activity, acting on carbon-nit...",3,0,0.03,58,1.000,0.05528,0.05528


In [1]:
#Essentially it goes like this:


# All genes in the genome
allGenesBed <- read.table("/Users/milanmalinsky/mz11-singleCoverGeneNames_GOBed.txt",as.is=TRUE)

# All genes in HDRs - i.e. the ones I am testing
allHDRGenesBed <- read.table("/Users/milanmalinsky/Sanger_work/Massoko_GO_analysis/all_HDR_intersect.bed",as.is=TRUE)

# Run the ontology analysis
HDR_o <- doOntology(allGenesBed,allHDRGenesBed,5)

doOntology <- function(allGenesBed, sigGenesBed, minNode) {
        geneUniverse <- unique(allGenesBed[,4])
        sigGenes <- unique(sigGenesBed[,4])
        testList <- factor(as.integer(geneUniverse %in% sigGenes))
        names(testList) <- unique(allGenesBed[,4])
        GOdata <- new("topGOdata", ontology = "MF", allGenes = testList, annot = annFUN.org, mapping = "org.Dr.eg.db", ID = "entrez", nodeSize=minNode)
        GOdataCC <- new("topGOdata", ontology = "CC", allGenes = testList, annot = annFUN.org, mapping = "org.Dr.eg.db", ID = "entrez", nodeSize=minNode)
        GOdataBP <- new("topGOdata", ontology = "BP", allGenes = testList, annot = annFUN.org, mapping = "org.Dr.eg.db", ID = "entrez", nodeSize=minNode)
        resultFisher <- runTest(GOdata, algorithm = "classic", statistic = "fisher")
        resultFisherWeigth <- runTest(GOdata, algorithm = "weight", statistic = "fisher")
        allRes <- GenTable(GOdata, classic = resultFisher, weight = resultFisherWeigth, orderBy = "weight", ranksOf = "classic", topNodes = 20)
        resultFisherCC <- runTest(GOdataCC, algorithm = "classic", statistic = "fisher")
        resultFisherWeigthCC <- runTest(GOdataCC, algorithm = "weight", statistic = "fisher")
        allResCC <- GenTable(GOdataCC, classic = resultFisherCC, weight = resultFisherWeigthCC, orderBy = "weight", ranksOf = "classic", topNodes = 20)
        resultFisherBP <- runTest(GOdataBP, algorithm = "classic", statistic = "fisher")
        resultFisherWeigthBP <- runTest(GOdataBP, algorithm = "weight", statistic = "fisher")
        allResBP <- GenTable(GOdataBP, classic = resultFisherBP, weight = resultFisherWeigthBP, orderBy = "weight", ranksOf = "classic", topNodes = 20)
        list(allRes,allResCC,allResBP,GOdata,GOdataCC,GOdataBP)
}

#This gives both classic GO enrichment (i.e. chi-squared test), and the fancy “weighted” one you can read about in the topGO manual and the paper they refer to there. Let me know if you need more help.

[1] 2

In [25]:
p = c(0.1,0.001,0.002,0.0002,0.0001,0.0001)

In [26]:
p

[1] 1e-01 1e-03 2e-03 2e-04 1e-04 1e-04

In [27]:
p.adjust(p, method = 'holm')

[1] 1e-01 3e-03 4e-03 8e-04 6e-04 6e-04

In [28]:
p.adjust(p, method = 'hochberg')

[1] 1e-01 3e-03 4e-03 8e-04 5e-04 5e-04

In [29]:
p.adjust(p, method = 'BH')

[1] 0.1000 0.0015 0.0024 0.0004 0.0003 0.0003

In [20]:
sort(p)*length(p):1

[1] 6e-05 5e-04 8e-04 3e-03 4e-03 1e-01

In [30]:
z <- c(39.389401808000002, 13.42335656, 12.668339525999999, 9.3463560799999996, 10.21835716, 8.6303431639999992,
       71.556839025000002, 72.887639234999995, 49.577837178000003, 55.264832901999995, 52.432831846999996,
       19.326519536999999, 13.791514039000001, 6.6784930560000006, 3.8844601560000003, 16.685636462000002,
       7.5685843210000003, 16.57363939, 6.8045909499999997, 13.691577735999999, 57.454610033999998, 13.892585239000001,
       13.806579897000001)

In [31]:
pvalue2sided=2*pnorm(-abs(z))

In [32]:
pvalue2sided

[1] 0.000000e+00 4.412651e-41 8.856938e-37 9.071938e-21 1.640983e-24
 [6] 6.116817e-18 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
[11] 0.000000e+00 3.213803e-83 2.866794e-43 2.414117e-11 1.025575e-04
[16] 1.667197e-62 3.773134e-14 1.080816e-61 1.013368e-11 1.140066e-42
[21] 0.000000e+00 7.025310e-44 2.326161e-43